In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bitsandbytes accelerate transformers langgraph

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="env")
hf_token = os.getenv("HF_TOKEN")

In [ ]:
# === Open Source Granite 8B model LLM ===
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    llm_int8_enable_fp32_cpu_offload=True
)
try:
    # Granite 8B model name on Hugging Face
    GRANITE_MODEL = "ibm-granite/granite-3.3-8b-instruct"

    # Load the model + tokenizer
    granite_model = AutoModelForCausalLM.from_pretrained(
        GRANITE_MODEL,
        device_map="auto",
        quantization_config=bnb_config  # Optional
    )

    granite_tokenizer = AutoTokenizer.from_pretrained(GRANITE_MODEL)

    # Create pipeline
    granite_pipe = pipeline(
        "text-generation",
        model=granite_model,
        tokenizer=granite_tokenizer,
        pad_token_id=granite_tokenizer.eos_token_id,
        return_full_text=False
    )
except Exception as e:
    print("[Warning] Failed to load LLaMA model:", e)
    granite_pipe = lambda prompt, **kwargs: [{"generated_text": "[granit model unavailable]"}]
granite_pipe = pipeline("text-generation", model=granite_model, tokenizer=granite_tokenizer)


In [ ]:
import os
import zipfile
import pickle
import numpy as np

# === Path to the ZIP file in Google Drive ===
drive_zip_path = "/content/drive/MyDrive/Portfolio datasets/vision/saadhaxxan_germantrafficsigns_kaggle.zip"
extract_path = "/content/gtsrb_extracted"

# === Create extraction directory if it doesn't exist ===
os.makedirs(extract_path, exist_ok=True)

# === Unzip the file ===
with zipfile.ZipFile(drive_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"✅ Unzipped into: {extract_path}")

# === List all files in the extracted folder ===
print("📂 Files:")
print(os.listdir(extract_path))

# === Function to load GTSRB Pickle files ===
def load_gtsrb_pickle(filename):
    with open(os.path.join(extract_path, filename), mode='rb') as f:
        data = pickle.load(f)
    X = data['features']  # images, shape: (N, 32, 32, 3)
    y = data['labels']    # list of class indices
    return X, y

# === Load test set ===
X_test, y_test = load_gtsrb_pickle("test.p")

# === Print dataset shapes ===
print(f"Test : {X_test.shape}, Labels: {len(y_test)}")


# AI Agents for Traffic Sign Classification and Explanation

This script uses two AI agents in a LangGraph pipeline:

- **ClassifyImage Agent**: Uses a pretrained `MobileNetV2` model to classify a traffic sign image from the GTSRB dataset.
- **ExplainAgent**: Uses `granite_pipe` to generate a **short natural-language explanation** (max 7 words) for the predicted sign.

The agents communicate via a shared `AgentState`, and the system runs on 5 random test images.

The explanation prompt is carefully crafted to ensure concise output:
> `'Sign name' sign – explain in plain sentence, max 7 words, no bullets, no numbers.`

This enables interpretable and human-readable AI behavior in traffic sign recognition.


# Short explanation on 5 words - can save tokens

In [42]:
# === Imports ===
import os
import pickle
import numpy as np
import random
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from langgraph.graph import StateGraph
from typing import TypedDict

# === Class labels (43 classes) ===
class_labels = [
    'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 50', 'Speed Limit 60',
    'Speed Limit 70', 'Speed Limit 80', 'End of Speed Limit 80', 'Speed Limit 100',
    'Speed Limit 120', 'No overtaking', 'No overtaking for trucks', 'Right-of-way at intersection',
    'Priority road', 'Yield', 'Stop Sign', 'No vehicles', 'No trucks', 'No entry',
    'General caution', 'Dangerous curve left', 'Dangerous curve right', 'Double curve',
    'Bumpy road', 'Slippery road', 'Road narrows', 'Construction', 'Traffic signals',
    'Pedestrians', 'Children crossing', 'Bicycles crossing', 'Snow', 'Animals',
    'End of all restrictions', 'Turn right ahead', 'Turn left ahead', 'Ahead only',
    'Go straight or right', 'Go straight or left', 'Keep right', 'Keep left',
    'Roundabout', 'End of no overtaking', 'End of no overtaking for trucks'
]

# === Load model ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(weights=None)
model.classifier[1] = nn.Linear(model.last_channel, len(class_labels))
model_path = "/content/drive/MyDrive/Models/mobilenet_43classes_f1_0.915.pt"
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# === Image transform (same as training)
transform = transforms.Compose([
    transforms.ToTensor()
])

# === Load test.p ===
with open("/content/gtsrb_extracted/test.p", "rb") as f:
    test_data = pickle.load(f)

X = test_data["features"]
y = test_data["labels"]

# === LangGraph State ===
class AgentState(TypedDict):
    image_index: int
    model_prediction: str
    original_label: str
    explanation: str

# === AI Agent: classify image using MobileNetV2 ===
def classify_image_agent(state: AgentState) -> AgentState:
    idx = state["image_index"]
    img_array = X[idx]
    label_id = y[idx]
    original_label = class_labels[label_id]

    image = Image.fromarray(img_array).convert("RGB").resize((64, 64))
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        pred_id = output.argmax(dim=1).item()
        predicted_label = class_labels[pred_id]

    return {
        "image_index": idx,
        "model_prediction": predicted_label,
        "original_label": original_label,
        "explanation": ""  # will be filled in next step
    }

# === AI Agent: generate explanation using granite_pipe ===
def explain_agent(state: AgentState) -> AgentState:
    prompt = f"""'{state['model_prediction']}' sign – explain in plain sentence, max 7 words, no bullets, no numbers, English only."""
    explanation = granite_pipe(prompt, max_new_tokens=15)[0]['generated_text']
    state["explanation"] = explanation.strip()
    return state


# === LangGraph definition ===
graph = StateGraph(AgentState)
graph.add_node("ClassifyImage", classify_image_agent)
graph.add_node("ExplainAgent", explain_agent)
graph.set_entry_point("ClassifyImage")
graph.add_edge("ClassifyImage", "ExplainAgent")
graph.set_finish_point("ExplainAgent")
graph = graph.compile()

# === Run agent on 5 random images ===
indices = random.sample(range(len(X)), 5)

for idx in indices:
    result = graph.invoke({"image_index": idx})
    print("\n=== Traffic Sign AI Agent Result ===")
    print(f"📷 Image Index     : {result['image_index']}")
    print(f"🏷️ Model Prediction: {result['model_prediction']}")
    print(f"📘 Original Label  : {result['original_label']}")
    print(f"🧠 Explanation     : {result['explanation']}")



=== Traffic Sign AI Agent Result ===
📷 Image Index     : 7391
🏷️ Model Prediction: Ahead only
📘 Original Label  : Ahead only
🧠 Explanation     : This sign warns drivers to keep right, avoiding intersections

=== Traffic Sign AI Agent Result ===
📷 Image Index     : 9229
🏷️ Model Prediction: No overtaking for trucks
📘 Original Label  : No overtaking for trucks
🧠 Explanation     : "Trucks cannot pass other vehicles on this road."

=== Traffic Sign AI Agent Result ===
📷 Image Index     : 9036
🏷️ Model Prediction: Children crossing
📘 Original Label  : Children crossing
🧠 Explanation     : 'Warning: Children may be crossing.'

## Instruction

=== Traffic Sign AI Agent Result ===
📷 Image Index     : 5592
🏷️ Model Prediction: Speed Limit 50
📘 Original Label  : Speed Limit 50
🧠 Explanation     : "Drive at a maximum of fifty within this zone."

=== Traffic Sign AI Agent Result ===
📷 Image Index     : 3327
🏷️ Model Prediction: Yield
📘 Original Label  : Yield
🧠 Explanation     : A 'Yield' sign in

### Long Explanation with Steps Prompt

The prompt was modified to instruct the LLM to return a **clear, step-by-step explanation** for the predicted traffic sign. It generates **2–5 concise numbered steps** to guide the driver, ensuring structured and helpful instructions with no generic introductions.

In [43]:
# === Imports ===
import os
import pickle
import numpy as np
import random
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from langgraph.graph import StateGraph
from typing import TypedDict

# === Class labels (43 classes) ===
class_labels = [
    'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 50', 'Speed Limit 60',
    'Speed Limit 70', 'Speed Limit 80', 'End of Speed Limit 80', 'Speed Limit 100',
    'Speed Limit 120', 'No overtaking', 'No overtaking for trucks', 'Right-of-way at intersection',
    'Priority road', 'Yield', 'Stop Sign', 'No vehicles', 'No trucks', 'No entry',
    'General caution', 'Dangerous curve left', 'Dangerous curve right', 'Double curve',
    'Bumpy road', 'Slippery road', 'Road narrows', 'Construction', 'Traffic signals',
    'Pedestrians', 'Children crossing', 'Bicycles crossing', 'Snow', 'Animals',
    'End of all restrictions', 'Turn right ahead', 'Turn left ahead', 'Ahead only',
    'Go straight or right', 'Go straight or left', 'Keep right', 'Keep left',
    'Roundabout', 'End of no overtaking', 'End of no overtaking for trucks'
]

# === Load model ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(weights=None)
model.classifier[1] = nn.Linear(model.last_channel, len(class_labels))
model_path = "/content/drive/MyDrive/Models/mobilenet_43classes_f1_0.915.pt"
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# === Image transform (same as training)
transform = transforms.Compose([
    transforms.ToTensor()
])

# === Load test.p ===
with open("/content/gtsrb_extracted/test.p", "rb") as f:
    test_data = pickle.load(f)

X = test_data["features"]
y = test_data["labels"]

# === LangGraph State ===
class AgentState(TypedDict):
    image_index: int
    model_prediction: str
    original_label: str
    explanation: str

# === AI Agent: classify image using MobileNetV2 ===
def classify_image_agent(state: AgentState) -> AgentState:
    idx = state["image_index"]
    img_array = X[idx]
    label_id = y[idx]
    original_label = class_labels[label_id]

    image = Image.fromarray(img_array).convert("RGB").resize((64, 64))
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        pred_id = output.argmax(dim=1).item()
        predicted_label = class_labels[pred_id]

    return {
        "image_index": idx,
        "model_prediction": predicted_label,
        "original_label": original_label,
        "explanation": ""  # will be filled in next step
    }

# === AI Agent: generate explanation using granite_pipe ===
def explain_agent(state: AgentState) -> AgentState:
    prompt = f"""
      You are an expert traffic instructor.

      Given the traffic sign: '{state['model_prediction']}', explain clearly and step-by-step what a driver should do.

      Use a numbered list of 2 to 5 steps. Be concise and clear.

      Avoid introductions or disclaimers. Respond in English only.
      """

    explanation = granite_pipe(prompt, max_new_tokens=150)[0]['generated_text']
    state["explanation"] = explanation.strip()
    return state


# === LangGraph definition ===
graph = StateGraph(AgentState)
graph.add_node("ClassifyImage", classify_image_agent)
graph.add_node("ExplainAgent", explain_agent)
graph.set_entry_point("ClassifyImage")
graph.add_edge("ClassifyImage", "ExplainAgent")
graph.set_finish_point("ExplainAgent")
graph = graph.compile()

# === Run agent on 5 random images ===
indices = random.sample(range(len(X)), 5)

for idx in indices:
    result = graph.invoke({"image_index": idx})
    print("\n=== Traffic Sign AI Agent Result ===")
    print(f"🏷️ Model Prediction: {result['model_prediction']}")
    print(f"📘 Original Label  : {result['original_label']}")
    explanation_lines = "\n".join(result["explanation"].splitlines())
    print(f"🧠 Explanation     :\n{explanation_lines}")



=== Traffic Sign AI Agent Result ===
🏷️ Model Prediction: Right-of-way at intersection
📘 Original Label  : Right-of-way at intersection
🧠 Explanation     :
1. Approach the intersection slowly and cautiously, observing all traffic signs and signals.
2. Identify if you have a stop sign or a yield sign. If you do, come to a complete stop before the stop line or yield line.
3. Check for oncoming vehicles and pedestrians crossing or waiting to cross.
4. Proceed with caution when it is safe to do so, ensuring that no other vehicles or pedestrians are crossing or entering the intersection.
5. Continue through the intersection, maintaining a safe speed and staying in your lane.

=== Traffic Sign AI Agent Result ===
🏷️ Model Prediction: End of Speed Limit 80
📘 Original Label  : End of Speed Limit 80
🧠 Explanation     :
1. Upon seeing the 'End of Speed Limit 80' sign, the driver should immediately prepare to reduce their speed gradually.
2. Locate the new speed limit sign ahead, which will indi